In [8]:
import os
import rasterio
import numpy as np

# =========================================================
# Diccionario representativo de los valores de clasificación
# =========================================================
CLASIFICACION = {
    0.0: "No bosque",
    0.5: "Vegetación / Transición",
    1.0: "Bosque denso"
}

# =========================================================
# Función para convertir intensidad a dB
# =========================================================
def to_db(array):
    return 10 * np.log10(array.clip(min=1e-6))

# =========================================================
# Umbrales (pueden ajustarse según Colombia)
# =========================================================
UMBRAL_BOSQUE_R = -13     # VH - VV mayor a esto -> bosque
UMBRAL_TRANSICION_R = -18 # entre -18 y -13 -> transición

UMBRAL_BOSQUE_VH = -16    # VH_dB > -16 dB -> bosque
UMBRAL_TRANSICION_VH = -20

# =========================================================
# Ruta base donde están las carpetas de fechas
# =========================================================
BASE_DIR = "deforestation"

# =========================================================
# Procesamiento por carpetas de fecha
# =========================================================

for folder in os.listdir(BASE_DIR):
    folder_path = os.path.join(BASE_DIR, folder)

    if not os.path.isdir(folder_path):
        continue

    # Buscar VV y VH dentro de la carpeta
    vv_path = None
    vh_path = None

    for f in os.listdir(folder_path):
        fp = os.path.join(folder_path, f)
        if "VV" in f.upper():
            vv_path = fp
        elif "VH" in f.upper():
            vh_path = fp

    if not vv_path or not vh_path:
        print(f"[WARN] Saltando {folder}, no se encontraron VV y VH")
        continue

    print(f"[INFO] Procesando carpeta: {folder}")

    # Leer VV y VH
    with rasterio.open(vv_path) as src_vv:
        VV = src_vv.read(1)
        meta = src_vv.meta.copy()

    with rasterio.open(vh_path) as src_vh:
        VH = src_vh.read(1)

    # Convertir a dB
    VV_dB = to_db(VV)
    VH_dB = to_db(VH)

    # Índice radar (muy útil)
    R = VH_dB - VV_dB

    # Clasificación
    clasif = np.zeros(VV.shape, dtype=np.float32)

    # Bosque denso
    bosque_mask = (VH_dB > UMBRAL_BOSQUE_VH) & (R > UMBRAL_BOSQUE_R)
    clasif[bosque_mask] = 1.0

    # Transición
    trans_mask = ((VH_dB > UMBRAL_TRANSICION_VH) &
                  (R > UMBRAL_TRANSICION_R) &
                  ~bosque_mask)
    clasif[trans_mask] = 0.5

    # No bosque (0) ya está inicializado

    # Guardar el raster de salida
    out_path = os.path.join(folder_path, f"classificado_{folder}.tif")
    meta.update(dtype="float32", count=1)

    with rasterio.open(out_path, "w", **meta) as dst:
        dst.write(clasif, 1)

    print(f"[OK] Archivo generado: {out_path}")


[INFO] Procesando carpeta: 2022-12-20
[OK] Archivo generado: deforestation\2022-12-20\classificado_2022-12-20.tif
[INFO] Procesando carpeta: 2023-12-27
[OK] Archivo generado: deforestation\2023-12-27\classificado_2023-12-27.tif
[INFO] Procesando carpeta: 2024-12-21
[OK] Archivo generado: deforestation\2024-12-21\classificado_2024-12-21.tif
[INFO] Procesando carpeta: 2025-10-29
[OK] Archivo generado: deforestation\2025-10-29\classificado_2025-10-29.tif


In [11]:
import geopandas as gpd

# Ruta a tu shapefile
path = r"C:\Users\dorito\Downloads\poligono.shp"

# Leer el shapefile
gdf = gpd.read_file(path)

# Obtener el AOI (suponiendo que es un único polígono)
aoi = gdf.geometry.unary_union  # Fusiona todo en una sola geometría si hay más de uno

print(aoi)


POLYGON ((-75.75002457264095 5.249970902786264, -75.37206874448688 5.24974125702263, -75.36428531535621 4.899977359765108, -75.75003261208246 4.89950077388499, -75.75002457264095 5.249970902786264))


C:\Users\dorito\AppData\Local\Temp\ipykernel_28120\2996481958.py:10: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  aoi = gdf.geometry.unary_union  # Fusiona todo en una sola geometría si hay más de uno


In [13]:
ee.Authenticate()


Successfully saved authorization token.


In [23]:
import os
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np

# -------------------------------
# Helper: load & resample a band
# -------------------------------
def load_band(path, target_profile=None):
    with rasterio.open(path) as src:
        if target_profile is None:
            return src.read(1), src.profile
        
        # Reproject to match target resolution and size
        data = np.zeros((target_profile["height"], target_profile["width"]), dtype=src.meta["dtype"])
        
        reproject(
            source=rasterio.band(src, 1),
            destination=data,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=target_profile["transform"],
            dst_crs=target_profile["crs"],
            resampling=Resampling.bilinear,
        )
        
        return data, None

# -------------------------------
# Compute index safely
# -------------------------------
def index_calc(numerator, denominator):
    return np.where(
        denominator == 0,
        0,
        numerator / denominator
    )

# -------------------------------
# MAIN FUNCTION
# -------------------------------
def compute_indices(input_folder, output_folder):
    # Sentinel-2 band mapping
    band_paths = {
        "B02": None,  # Blue 10m
        "B03": None,  # Green 10m
        "B04": None,  # Red 10m
        "B05": None,  # Red Edge 1 (20m)
        "B08": None,  # NIR 10m
        "B12": None,  # SWIR 20m
    }

    # Detect band files
    for f in os.listdir(input_folder):
        for key in band_paths:
            if key in f:
                band_paths[key] = os.path.join(input_folder, f)

    print("\nBandas encontradas:")
    print(band_paths)

    # Fallback except errors
    if any(v is None for v in band_paths.values()):
        raise ValueError("Faltan bandas necesarias (B02, B03, B04, B05, B08, B12)")

    # Load 10m reference band
    nir, profile_10m = load_band(band_paths["B08"])
    profile_out = profile_10m.copy()
    profile_out.update(dtype="float32", nodata=0, compress="lzw")

    # Load other bands resampled to 10m
    red, _ = load_band(band_paths["B04"], profile_10m)
    green, _ = load_band(band_paths["B03"], profile_10m)
    blue, _ = load_band(band_paths["B02"], profile_10m)
    red_edge, _ = load_band(band_paths["B05"], profile_10m)
    swir, _ = load_band(band_paths["B12"], profile_10m)

    # -------------------------------
    # Índices
    # -------------------------------
    ndvi = index_calc(nir - red, nir + red)
    ndre = index_calc(nir - red_edge, nir + red_edge)
    ndwi = index_calc(green - nir, green + nir)
    evi = 2.5 * ( (nir - red) / (nir + 6*red - 7.5*blue + 1) )
    savi = 1.5 * ((nir - red) / (nir + red + 0.5))

    # -------------------------------
    # Save outputs
    # -------------------------------
    os.makedirs(output_folder, exist_ok=True)
    outputs = {
        "NDVI.tif": ndvi,
        "NDRE.tif": ndre,
        "NDWI.tif": ndwi,
        "EVI.tif": evi,
        "SAVI.tif": savi,
    }

    for name, data in outputs.items():
        out_path = os.path.join(output_folder, name)
        with rasterio.open(out_path, "w", **profile_out) as dst:
            dst.write(data.astype("float32"), 1)
        print(f"✔️ Guardado: {out_path}")

    print("\nTodo listo.")

# -------------------------------
# Ejecutar
# -------------------------------
if __name__ == "__main__":
    compute_indices(
        input_folder=r"./2025-11-18 indices/",
        output_folder=r"./2025-11-18 indices/"
    )



Bandas encontradas:
{'B02': './2025-11-18 indices/B02_10m.tif', 'B03': './2025-11-18 indices/B03_10m.tif', 'B04': './2025-11-18 indices/B04_10m.tif', 'B05': './2025-11-18 indices/B05_20m.tif', 'B08': './2025-11-18 indices/B08_10m.tif', 'B12': './2025-11-18 indices/B12_20m.tif'}


C:\Users\dorito\AppData\Local\Temp\ipykernel_28120\4259350695.py:36: RuntimeWarning: invalid value encountered in divide
  numerator / denominator
C:\Users\dorito\AppData\Local\Temp\ipykernel_28120\4259350695.py:84: RuntimeWarning: divide by zero encountered in divide
  evi = 2.5 * ( (nir - red) / (nir + 6*red - 7.5*blue + 1) )


✔️ Guardado: ./2025-11-18 indices/NDVI.tif
✔️ Guardado: ./2025-11-18 indices/NDRE.tif
✔️ Guardado: ./2025-11-18 indices/NDWI.tif
✔️ Guardado: ./2025-11-18 indices/EVI.tif
✔️ Guardado: ./2025-11-18 indices/SAVI.tif

Todo listo.
